# ME599 - AI in Clean Energy Watt's Next
Team Members: Ricardo Lee, Evan Muschler, Vinh Nguyen, Noah Tanner

## 1. Installation of Requisite Libraries
Installing the full AeroSandbox package, which contains NeuralFoil and methods for generating parameters from airfoil coordinates.

In [ ]:
!pip install aerosandbox[full]
!pip install pyDOE

## 2. Import Libraries

In [2]:
import aerosandbox as asb
import aerosandbox.numpy as np
import matplotlib.pyplot as plt
import aerosandbox.tools.pretty_plots as p

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from scipy.stats import qmc
from scipy.spatial import distance
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize
from scipy.stats import norm
from pyDOE import lhs
import os
from scipy.stats.qmc import Sobol
from itertools import product
import seaborn as sns

In [ ]:
# Testing the packaged airfoil generation code. First makes a regular airfoil from airfoil database.
coordinate_airfoil = asb.Airfoil("dae11")
coordinate_airfoil
fig, ax = plt.subplots(figsize=(6, 2))
coordinate_airfoil.draw()
# Converts airfoil to kulfan parameterized airfoil. It looks the same, but the dots aren't plotted so the figure looks smoother. 
kulfan_airfoil = coordinate_airfoil.to_kulfan_airfoil()
kulfan_airfoil
fig, ax = plt.subplots(figsize=(6, 2))
kulfan_airfoil.draw()
# prints parameters so we can see how data is formatted
print(kulfan_airfoil.kulfan_parameters)

In [ ]:
# test to just see how we can play around with airfoil shapes based on kulfan parameters
test = asb.KulfanAirfoil(
    name="Test",
    lower_weights=[-1, -.5, 0, 0, -1, .5, -1, -.75],
    upper_weights=1.0 * np.ones(8),
    leading_edge_weight=.9,
    TE_thickness=.005,
)
print(test.kulfan_parameters)
fig, ax = plt.subplots(figsize=(6, 2))
test.draw()

## 3. Define Neural Foil Helper Functions
Here, we define helper functions that help us put the Kulfan Parameters into a Bayesian Optimization-friendly state. The Kulfan parameters are by default in a dictionary, but we want them in a single list format for easy data manipulation. 

Also makes helper functions that can make the next foils to try based on the new parameters chosen by Bayesian Optimization.

In [5]:
from neuralfoil.main import get_aero_from_kulfan_parameters

# This converts the dictionary of Kulfan parameters into a single list. 
def kulfan_to_bo(kf_params):
    list_lower = kf_params["lower_weights"]
    list_upper = kf_params["upper_weights"]
    leading_edge = kf_params["leading_edge_weight"]
    trailing_edge = kf_params["TE_thickness"]
    result = [leading_edge, trailing_edge, *list_upper, *list_lower]
    return result

# This converts the list of Bayesian Optimization parameters back into a Kulfan parameter dictionary for the AeroSanbox library
def bo_to_kulfan(bo):
    result = {"leading_edge_weight": bo[0],
              "TE_thickness": bo[1],
              "upper_weights": bo[2:9],
              "lower_weights": bo[10:17]}
    return result

# Helper function to generate airfoils from given Kulfan parameters. Mostly there to help automate the code during BO process
def make_foil_kf(kf, name):
    foil = asb.KulfanAirfoil(
    name=name,
    lower_weights=kf["lower_weights"],
    upper_weights=kf["upper_weights"],
    leading_edge_weight=kf["leading_edge_weight"],
    TE_thickness=kf["TE_thickness"],
)
    return foil

# makes Kulfan airfoils from just (X,Y) coordinates from the BO optimization or .dat file.
def make_foil_coords(coords, name):
    foil = asb.Airfoil(name = name, coords = coords)
    kf_foil = foil.to_kulfan_airfoil()
    return kf_foil

# generate aero performance variable containing CL and CD over variable angle of attack and reynolds numbers from kulfan parameterized foil
def get_CL_CD_perf(kf_foil):
    # define angles of attack range over which to collect values, these are hardcoded for now
    alpha_start = 1
    alpha_end = 20
    alpha = np.linspace(alpha_start,alpha_end,200)

    # define reynolds number range over which to collect values, these are hardcoded for now
    Re_start = 1e6
    Re_end = 10e6
    Re_list = np.linspace(Re_start, Re_end, 10) 

    # temporary storage lists
    Re_all = []
    alpha_all = []
    CL_all = []
    CD_all = []
    
    # loop over Reynolds numbers
    for Re in Re_list:
        aero_ = get_aero_from_kulfan_parameters(
            kulfan_parameters = kf_foil,
            alpha=alpha,  # NeuralFoil expects degrees
            Re=Re,
            model_size="xxlarge"
        )
    
        Re_all.extend([Re] * len(alpha))
        alpha_all.extend(alpha)
        CL_all.extend(aero_["CL"])
        CD_all.extend(aero_["CD"])
    
    # final neuralfoil aero_performance dictionary as NumPy arrays, with alpha in radians for the BEM code
    aero_performance = {
        "Re": np.array(Re_all, dtype=np.float64),
        "alpha": np.array(np.radians(alpha_all), dtype=np.float64),
        "CL": np.array(CL_all, dtype=np.float64),
        "CD": np.array(CD_all, dtype=np.float64)
    }

    return aero_performance

## 5. Define a Bayesian Optimization Model
Defines a Bayesian Optimization Model to optimize Cp given the 18 Kulfan parameters

### 5.1 Define conditions for optimization
What are the conditions that we are trying to optimize our airfoil for? Here, we set wind velocity, Reynolds number ($Re$), Mach number ($M_{inf}$), the hub radius of the wind turbine ($r_{hub}$), the number of blades ($n$), the total turbine radius ($r_t$), and a range for the angle of attack ($\alpha$).

In [ ]:

# Define Input turbine properties
U_design = 8  # Design wind speed [m/s]
R = 25  # Turbine radius [m]
eta_0 = 0.90  # Balance of system efficiency
lambda_design = 8  # Design tip speed ratio
B = 3  # Number of blades
R_hub = 2  # Hub radius [m]
N = 50  # Number of blade segments
rho = 1.225  # Air density [kg/m^3]
nu = 1.511e-5  # Kinematic viscosity of air at 20 C [m^2/s]
Re_design = 1.5e6 # Starting guess on Reynolds number (1.5e6 is decent)
m_inf = 0.2 # currently not used... need to resolve.... <--------------------------
alpha_range = [5, 15]

def get_basic_kulfan():
    return {
        "leading_edge_weight": 0.2,      # Moderate leading edge radius
        "TE_thickness": 0.02,            # Typical trailing edge thickness
        "upper_weights": [0.2, 0.2, 0.15, 0.1, 0.05, 0.0, -0.05, -0.1],  # Cambered upper surface
        "lower_weights": [-0.1, -0.1, -0.08, -0.06, -0.04, -0.02, 0.0, 0.02]  # Typical lower surface
    }

initial_kulfan = get_basic_kulfan()
print(initial_kulfan)
starting_foil = asb.KulfanAirfoil(
    name="Initial Foil",
    lower_weights=initial_kulfan["lower_weights"],
    upper_weights=initial_kulfan["upper_weights"],
    leading_edge_weight=initial_kulfan["leading_edge_weight"],
    TE_thickness=initial_kulfan["TE_thickness"]
)

starting_foil.draw()



In [ ]:
from bayesianOpModel import bo_loop 
from bayopTest import bayop_test
#best_design, best_kf, best_foil = bo_loop(re, alpha)
kulfan_best, max_score = bayop_test( U_design, R, eta_0, lambda_design, B, R_hub, N, initial_kulfan, Re_design, rho=rho, nu=nu, alpha_range=alpha_range)

In [ ]:
optimized_foil = asb.KulfanAirfoil(
    name="optimized_airfoil",
    lower_weights=kulfan_best["lower_weights"],
    upper_weights=kulfan_best["upper_weights"],
    leading_edge_weight=kulfan_best["leading_edge_weight"],
    TE_thickness=kulfan_best["TE_thickness"]
)

optimized_foil.draw()


### 5.4 

## 6. Data Visualization
Function to visualize how the airfoil changes shape as it goes through the Bayesian Optimization process

In [ ]:
from animate_optimization import create_optimization_animation
optimization_history_path= 'optimization_history_20250601_144301'

create_optimization_animation(optimization_history_path, output_file='optimization.gif')